### Imports

In [156]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor
from tqdm.contrib.concurrent import thread_map
pd.set_option('display.max_colwidth', None)

### GET AIRLINES URLS ###

In [157]:
url = 'https://www.pilotjobsnetwork.com/'
max_urls = 5

def get_airlines_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href and 'jobs/' in href:
            airline_url = url + href
            airline_name = link.text.strip()
            yield airline_url, airline_name # Generator creation

### GET INFO FROM PAGES ###

In [158]:
def get_airline_tables(airline_tuple):
    airline_url, airline_name = airline_tuple
    try:
        list_tables = pd.read_html(airline_url)
        salary = list_tables[3].iloc[2:4,1].tolist()
        # iloc iloc[2:4,2] gets you the dates, but remove the salary ... smthing to 
        last_update = list_tables[3].iloc[2:4,2].tolist()
        return [airline_url, airline_name] + salary + last_update
    except Exception as e:
        print(e)
        try:
            return list_tables[3]
        except NameError:
            return airline_url

### MAP IT !! ###

In [159]:
list_salaries = list(thread_map(get_airline_tables, get_airlines_urls(url)))
# print dans un excel le list_salaries

0it [00:00, ?it/s]

'ascii' codec can't encode characters in position 18-19: ordinal not in range(128)
URL can't contain control characters. '/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google' (found at least '\r')


### DF AS OUTPUT ###

In [160]:
df_salary = pd.DataFrame([salary for salary in list_salaries if type(salary) == list])
df_salary

0  \
0          https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH   
1    https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines   
2    https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd   
3           https://www.pilotjobsnetwork.com/jobs/Avion_Express   
4       https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines   
..                                                          ...   
494            https://www.pilotjobsnetwork.com/jobs/FlyinGroup   
495           https://www.pilotjobsnetwork.com/jobs/MS_AVIATION   
496         https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation   
497      https://www.pilotjobsnetwork.com/jobs/Compass_Airlines   
498        https://www.pilotjobsnetwork.com/jobs/Luxaviation_UK   

                                                1  \
0            7Mar - Germany (Ma) - Eurowings GmbH   
1    7Mar - Hong Kong (Ma) - Greater Bay Airlines   
2        6Mar - Malta (Ma) - Eurowings Europe Ltd   
3           6Mar - Lithuania (Ch) - Avion Express   
4          5Mar - Turkey (Ma) - Corendon Airlines   
..                                            ...   
494             13Aug - Belgium (Fr) - FlyinGroup   
495            12Aug - Austria (Fr) - MS AVIATION   
496           12Aug - France (Re) - ESMA Aviation   
497            9Aug - USA (Ma) - Compass Airlines   
498               3Aug - UK (Ch) - Luxaviation UK   

                                                                                                                                                                   2  \
0           14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)   
1                                                                                                                                                                NaN   
2    AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment)   
3                                          Captains 5500 EUR basic pay + 50 EUR/BH. Loyalty bonus 2500 EUR paid twice per year (applicable from 2nd year of service)   
4                                                                                                                                                               6250   
..                                                                                                                                                               ...   
494                                                                                                                                                              NaN   
495                                                                                                                                                              NaN   
496                                                                                                                                                              NaN   
497                                                                                                                                                              NaN   
498                                                                                                                            GBP53,200 + loyalty bonus (see above)   

                                                                                                                                                                                                         3  \
0    8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade.   
1                                                                                                                                                                                           $80

### ERRORS LIST ###

In [161]:
[salary for salary in list_salaries if type(salary) == str]

['https://www.pilotjobsnetwork.com/jobs/Air_CaraÃ¯bes_Atlantique',
 'https://www.pilotjobsnetwork.com/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google']

### DF REWORK ###

In [162]:
# First name columns
df_salary.columns = ["URL", "AirlineName", "CaptMax", "CaptMin", "DateCaptMax", "DateCaptMin"]
df_salary

URL  \
0          https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH   
1    https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines   
2    https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd   
3           https://www.pilotjobsnetwork.com/jobs/Avion_Express   
4       https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines   
..                                                          ...   
494            https://www.pilotjobsnetwork.com/jobs/FlyinGroup   
495           https://www.pilotjobsnetwork.com/jobs/MS_AVIATION   
496         https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation   
497      https://www.pilotjobsnetwork.com/jobs/Compass_Airlines   
498        https://www.pilotjobsnetwork.com/jobs/Luxaviation_UK   

                                      AirlineName  \
0            7Mar - Germany (Ma) - Eurowings GmbH   
1    7Mar - Hong Kong (Ma) - Greater Bay Airlines   
2        6Mar - Malta (Ma) - Eurowings Europe Ltd   
3           6Mar - Lithuania (Ch) - Avion Express   
4          5Mar - Turkey (Ma) - Corendon Airlines   
..                                            ...   
494             13Aug - Belgium (Fr) - FlyinGroup   
495            12Aug - Austria (Fr) - MS AVIATION   
496           12Aug - France (Re) - ESMA Aviation   
497            9Aug - USA (Ma) - Compass Airlines   
498               3Aug - UK (Ch) - Luxaviation UK   

                                                                                                                                                             CaptMax  \
0           14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)   
1                                                                                                                                                                NaN   
2    AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment)   
3                                          Captains 5500 EUR basic pay + 50 EUR/BH. Loyalty bonus 2500 EUR paid twice per year (applicable from 2nd year of service)   
4                                                                                                                                                               6250   
..                                                                                                                                                               ...   
494                                                                                                                                                              NaN   
495                                                                                                                                                              NaN   
496                                                                                                                                                              NaN   
497                                                                                                                                                              NaN   
498                                                                                                                            GBP53,200 + loyalty bonus (see above)   

                                                                                                                                                                                                   CaptMin  \
0    8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade.   
1                                                                                                                                                                                           $

### PIVOT TABLE

In [166]:
df_stack = (df_salary.set_index(['URL', 'AirlineName', 'DateCaptMax', 'DateCaptMin'])
   .rename_axis(['Top/Base'], axis=1)
   .stack(dropna=False) # Put True to remove NaNs
   .reset_index())
df_stack.columns = ['URL', 'AirlineName', 'DateCaptMax', 'DateCaptMin', 'Top/Base', 'Salary']
df_stack

URL  \
0          https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH   
1          https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH   
2    https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines   
3    https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines   
4    https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd   
..                                                          ...   
993         https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation   
994      https://www.pilotjobsnetwork.com/jobs/Compass_Airlines   
995      https://www.pilotjobsnetwork.com/jobs/Compass_Airlines   
996        https://www.pilotjobsnetwork.com/jobs/Luxaviation_UK   
997        https://www.pilotjobsnetwork.com/jobs/Luxaviation_UK   

                                      AirlineName DateCaptMax DateCaptMin  \
0            7Mar - Germany (Ma) - Eurowings GmbH   15/Jan/23   15/Jan/23   
1            7Mar - Germany (Ma) - Eurowings GmbH   15/Jan/23   15/Jan/23   
2    7Mar - Hong Kong (Ma) - Greater Bay Airlines         NaN   22/Aug/22   
3    7Mar - Hong Kong (Ma) - Greater Bay Airlines         NaN   22/Aug/22   
4        6Mar - Malta (Ma) - Eurowings Europe Ltd    6/Mar/23    6/Mar/23   
..                                            ...         ...         ...   
993           12Aug - France (Re) - ESMA Aviation         NaN         NaN   
994            9Aug - USA (Ma) - Compass Airlines         NaN         NaN   
995            9Aug - USA (Ma) - Compass Airlines         NaN         NaN   
996               3Aug - UK (Ch) - Luxaviation UK    3/Nov/14   11/Jan/08   
997               3Aug - UK (Ch) - Luxaviation UK    3/Nov/14   11/Jan/08   

    Top/Base  \
0    CaptMax   
1    CaptMin   
2    CaptMax   
3    CaptMin   
4    CaptMax   
..       ...   
993  CaptMin   
994  CaptMax   
995  CaptMin   
996  CaptMax   
997  CaptMin   

                                                                                                                                                                                                    Salary  
0                                                 14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)  
1    8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade.  
2                                                                                                                                                                                                      NaN  
3                                                                                                                                                                                           $80000 HKD/MTH  
4                                          AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment)  
..                                                                                                                                                                                                     ...  
993                                                                                                                                                                                                    NaN  
994                                                                                                                                                                                                    NaN  
995                                                                                                                                                                                                    NaN  
996                                                                                    

### Select date (min/max) based on "CaptMin/CaptMax"

In [167]:
# Create a unique column for date depending on Top/Base column value. Extract Year
df_stack['Date_reworked'] = np.where(df_stack['Top/Base'] == 'CaptMax', df_stack['DateCaptMax'], df_stack['DateCaptMin'])
df_stack['Date_reworked'] = pd.to_datetime(df_stack['Date_reworked'])
df_stack['Year'] = df_stack['Date_reworked'].dt.year
df_stack['Year'] = df_stack['Year'].astype('Int64')


# Split column AirlineName on " - " seperator
df_stack[["DateFromLink", "Country", "Name"]] = df_stack["AirlineName"].str.split(" - ", expand=True)

# # Cleaning columns. Droping non necessary
clean_order = ['URL', 'Country', 'Name', 'Year', 'Top/Base', 'Salary']
df_stack = df_stack[clean_order]
df_stack.to_excel('airline_list.xlsx')
df_stack

URL  \
0          https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH   
1          https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH   
2    https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines   
3    https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines   
4    https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd   
..                                                          ...   
993         https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation   
994      https://www.pilotjobsnetwork.com/jobs/Compass_Airlines   
995      https://www.pilotjobsnetwork.com/jobs/Compass_Airlines   
996        https://www.pilotjobsnetwork.com/jobs/Luxaviation_UK   
997        https://www.pilotjobsnetwork.com/jobs/Luxaviation_UK   

            Country                  Name  Year Top/Base  \
0      Germany (Ma)        Eurowings GmbH  2023  CaptMax   
1      Germany (Ma)        Eurowings GmbH  2023  CaptMin   
2    Hong Kong (Ma)  Greater Bay Airlines  <NA>  CaptMax   
3    Hong Kong (Ma)  Greater Bay Airlines  2022  CaptMin   
4        Malta (Ma)  Eurowings Europe Ltd  2023  CaptMax   
..              ...                   ...   ...      ...   
993     France (Re)         ESMA Aviation  <NA>  CaptMin   
994        USA (Ma)      Compass Airlines  <NA>  CaptMax   
995        USA (Ma)      Compass Airlines  <NA>  CaptMin   
996         UK (Ch)        Luxaviation UK  2014  CaptMax   
997         UK (Ch)        Luxaviation UK  2008  CaptMin   

                                                                                                                                                                                                    Salary  
0                                                 14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)  
1    8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade.  
2                                                                                                                                                                                                      NaN  
3                                                                                                                                                                                           $80000 HKD/MTH  
4                                          AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment)  
..                                                                                                                                                                                                     ...  
993                                                                                                                                                                                                    NaN  
994                                                                                                                                                                                                    NaN  
995                                                                                                                                                                                                    NaN  
996                                                                                                                                                                  GBP53,200 + loyalty bonus (see above)  
997                                                                                                                                                                                              See above  

[998 rows x 6 columns]